**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
FOLDER = "annotation_encode_e2g_benchmark"

In [3]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    FOLDER)
for(fname in dir(fdiry)){print(fname)}

[1] "description.tsv"
[1] "K562.EPCrisprBenchmark.active.bed.gz"
[1] "K562.EPCrisprBenchmark.active.merge.bed.gz"
[1] "K562.EPCrisprBenchmark.active.region2TSS_Original.bed"
[1] "K562.EPCrisprBenchmark.active.region2TSS_Original.bedpe.gz"
[1] "K562.EPCrisprBenchmark.active.region2TSS_Original.bigInteract"
[1] "K562.EPCrisprBenchmark.active.region2TSS_Updated.bed"
[1] "K562.EPCrisprBenchmark.active.region2TSS_Updated.bedpe.gz"
[1] "K562.EPCrisprBenchmark.active.region2TSS_Updated.bigInteract"
[1] "K562.EPCrisprBenchmark.total.bed.gz"
[1] "K562.EPCrisprBenchmark.total.merge.bed.gz"
[1] "tmp"


## Import data

In [6]:
fdiry  = file.path(FD_RES, "results", "region", FOLDER)
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, show_col_types = FALSE)

dat_cnames = dat
print(dim(dat))
dat

[1] 12  2


Name,Description
<chr>,<chr>
Chrom,Chromosome of the perturbed element
Start,Start of the perturbed element (0-based)
End,End of the perturbed element
Name,"Identifier for the given E-G pair in the form of 'gene|chr:start-stop:strand', e.g. 'BET1L|chr11:220536-220704:*'"
Score,"Change in gene expression upon CRISPRi perturbation, e.g. precentage change in expression or log fold change"
Region,Location of the perturbed element
Gene_Symbol,target gene symbol
Gene_TSS,target gene TSS
Regulated,Whether it is postive case or negative case; True for postive case


In [7]:
### init: set column names
cnames = dat_cnames$Name
cnames = c("Chrom_ATAC", "Start_ATAC", "End_ATAC", cnames, "Overlap")

### init: set file path
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    FOLDER)
fname = paste("peak.annotation", ".total.bed.gz", sep="*")
fglob = file.path(fdiry, fname)
fpath = Sys.glob(fglob)

### init: annotation and label
annotation = "ENCODE-E2G"
label      = "ENCODE-E2G_Benchmark"

### read data
dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
dat = dat %>% dplyr::mutate(
    Peak_ATAC  = paste0(Chrom_ATAC, ":", Start_ATAC, "-", End_ATAC),
    Annotation = annotation,
    Label      = label,
    Region     = paste0(Chrom, ":", Start, "-", End),
    Value      = Score,
    Note       = Reference
)

### assign and show
dat_peak_annot_import = dat
print(dim(dat))
head(dat)

[1] 8678   21


Chrom_ATAC,Start_ATAC,End_ATAC,Chrom,Start,End,Name,Score,Region,Gene_Symbol,⋯,Regulated,Reference,Assay_Type,Assay_Label,Overlap,Peak_ATAC,Annotation,Label,Value,Note
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,⋯,<lgl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>
chr1,3773953,3776305,chr1,3774714,3775214,CEP104|chr1:3691278-3691778:*,-0.29343,chr1:3774714-3775214,CEP104,⋯,TRUE,Ulirsch2016,ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:CEP104,500,chr1:3773953-3776305,ENCODE-E2G,ENCODE-E2G_Benchmark,-0.29343,Ulirsch2016
chr1,3773953,3776305,chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:*,-0.33118,chr1:3774714-3775214,LRRC47,⋯,TRUE,Ulirsch2016,ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:LRRC47,500,chr1:3773953-3776305,ENCODE-E2G,ENCODE-E2G_Benchmark,-0.33118,Ulirsch2016
chr1,3773953,3776305,chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:*,-0.47202,chr1:3774714-3775214,SMIM1,⋯,TRUE,Ulirsch2016,ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1,500,chr1:3773953-3776305,ENCODE-E2G,ENCODE-E2G_Benchmark,-0.47202,Ulirsch2016
chr1,3803939,3806560,chr1,3803570,3805848,LRRC47|chr1:3720134-3722412:.,-0.00147,chr1:3803570-3805848,LRRC47,⋯,FALSE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:LRRC47,1909,chr1:3803939-3806560,ENCODE-E2G,ENCODE-E2G_Benchmark,-0.00147,"Gasperini et al., 2019"
chr1,3803939,3806560,chr1,3803570,3805848,SMIM1|chr1:3720134-3722412:.,0.02568,chr1:3803570-3805848,SMIM1,⋯,FALSE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1,1909,chr1:3803939-3806560,ENCODE-E2G,ENCODE-E2G_Benchmark,0.02568,"Gasperini et al., 2019"
chr1,4126790,4128217,chr1,4126791,4127291,SMIM1|chr1:4186851-4187351:.,0.02338,chr1:4126791-4127291,SMIM1,⋯,FALSE,"Gasperini et al., 2019",ENCODE-E2G_Benchmark,ENCODE-E2G_Benchmark:SMIM1,500,chr1:4126790-4128217,ENCODE-E2G,ENCODE-E2G_Benchmark,0.02338,"Gasperini et al., 2019"


## Arrange and summarize

In [13]:
### init
dat = dat_peak_annot_import

### summarize
dat = dat %>%
    dplyr::group_by(Chrom_ATAC, Start_ATAC, End_ATAC, Peak_ATAC, Annotation, Label) %>%
    dplyr::summarise(
        Count   = n(), 
        Region  = paste(Region, collapse="|"),
        Score_Mean = mean(Value),
        Score_Max  = max(Value),
        Score_Min  = min(Value),
        Note    = paste(sort(unique(Note)), collapse="|"),
        .groups = "drop")

## Arrange
dat = dat %>% 
    dplyr::arrange(Chrom_ATAC, Start_ATAC, End_ATAC) %>%
    dplyr::select(-Chrom_ATAC, -Start_ATAC, -End_ATAC) %>%
    dplyr::rename("Peak" = Peak_ATAC)

### assign and show
dat_peak_annot_summary = dat
print(dim(dat))
head(dat)

[1] 3007    9


Peak,Annotation,Label,Count,Region,Score_Mean,Score_Max,Score_Min,Note
<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
chr1:3773953-3776305,ENCODE-E2G,ENCODE-E2G_Benchmark,3,chr1:3774714-3775214|chr1:3774714-3775214|chr1:3774714-3775214,-0.3655433,-0.29343,-0.47202,Ulirsch2016
chr1:3803939-3806560,ENCODE-E2G,ENCODE-E2G_Benchmark,2,chr1:3803570-3805848|chr1:3803570-3805848,0.0121050,0.02568,-0.00147,"Gasperini et al., 2019"
chr1:4126790-4128217,ENCODE-E2G,ENCODE-E2G_Benchmark,1,chr1:4126791-4127291,0.0233800,0.02338,0.02338,"Gasperini et al., 2019"
chr1:5304675-5305858,ENCODE-E2G,ENCODE-E2G_Benchmark,1,chr1:5304578-5305078,0.0267200,0.02672,0.02672,"Gasperini et al., 2019"
chr1:8197559-8198606,ENCODE-E2G,ENCODE-E2G_Benchmark,1,chr1:8197448-8198244,-0.0198800,-0.01988,-0.01988,"Gasperini et al., 2019"
chr1:8857749-8858710,ENCODE-E2G,ENCODE-E2G_Benchmark,1,chr1:8858063-8858563,-0.1074100,-0.10741,-0.10741,"Gasperini et al., 2019"


## Explore and check results

In [14]:
dat = dat_peak_annot_summary
lst = split(dat, dat$Label)
lst = lapply(lst, function(dat){
    table(dat$Note)
})
lst

$`ENCODE-E2G_Benchmark`

                                      Fulco2016 
                                             79 
                            Fulco2016|Fulco2019 
                                              1 
     Fulco2016|Fulco2019|Gasperini et al., 2019 
                                              1 
                                      Fulco2019 
                                            435 
               Fulco2019|Gasperini et al., 2019 
                                             24 
                         Gasperini et al., 2019 
                                           1884 
               Gasperini et al., 2019|Huang2018 
                                              4 
               Gasperini et al., 2019|Klann2017 
                                              2 
       Gasperini et al., 2019|Klann2017|Xie2017 
                                              2 
Gasperini et al., 2019|Schraivogel et al., 2020 
                                            

In [15]:
dat = dat_peak_annot_summary
lst = split(dat, dat$Label)
lst = lapply(lst, function(dat){
    table(dat$Count)
})
lst

$`ENCODE-E2G_Benchmark`

   1    2    3    4    5    6    7    8    9   10   11   12   13   14   16   17 
1327  525  299  316  167  133   40   40   19   29   67   16    4    6    4    2 
  18   19   20   21   22   24   28   29 
   2    2    1    2    3    1    1    1 


## Save results

In [16]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "summary")
fname = "peak.summary.encode_e2g_benchmark.total.tsv"
fpath = file.path(fdiry, fname)

dat = dat_peak_annot_summary
write_tsv(dat, fpath)